In [1]:
from vampyr import vampyr3d as vp
from orbital4c import orbital as orb
from orbital4c import complex_fcn as cf
from orbital4c import operators as opr
import numpy as np
from scipy.linalg import eig, inv
import importlib
importlib.reload(orb)
from scipy.special import legendre, laguerre, erf, gamma
from scipy.special import gamma
from scipy.constants import hbar
import numpy.linalg as LA


c = 137   # NOT A GOOD WAY. MUST BE FIXED!!!
alpha = 1/c
k = -1
l = 0
n = 1
m = 0.5
Z = 2

mra = vp.MultiResolutionAnalysis(box=[-20,20], order=7)
prec = 1.0e-4
origin = [0.1, 0.2, 0.3]  # origin moved to avoid placing the nuclar charge on a node

In [2]:
a_coeff = 3.0
b_coeff = np.sqrt(a_coeff/np.pi)**3
gauss = vp.GaussFunc(b_coeff, a_coeff, origin)
gauss_tree = vp.FunctionTree(mra)
vp.advanced.build_grid(out=gauss_tree, inp=gauss)
vp.advanced.project(prec=prec, out=gauss_tree, inp=gauss)
gauss_tree.normalize()

In [3]:
orb.orbital4c.mra = mra
cf.complex_fcn.mra = mra
complexfc = cf.complex_fcn()
complexfc.copy_fcns(real=gauss_tree)

In [4]:
spinorb1 = orb.orbital4c()
spinorb1.copy_components(La=complexfc)
spinorb1.init_small_components(prec/10)
spinorb1.normalize()

spinorb2 = orb.orbital4c()
spinorb2.copy_components(Lb=complexfc)
spinorb2.init_small_components(prec/10)
spinorb2.normalize()

spinorb_vec = np.array([spinorb1,spinorb2])

In [5]:
def u(r):
    u = erf(r)/r + (1/(3*np.sqrt(np.pi)))*(np.exp(-(r**2)) + 16*np.exp(-4*r**2))
    #erf(r) is an error function that is supposed to stop the potential well from going to inf.
    #if i remember correctly
    return u

def V(x):
    r = np.sqrt(x[0]**2 + x[1]**2 + x[2]**2)
#    c = 0.0435
    c = 0.000435 # ten times tighter nuclear potential
    f_bar = u(r/c)/c
    return f_bar

Peps = vp.ScalingProjector(mra,prec)
f = lambda x: V([x[0]-origin[0],x[1]-origin[1],x[2]-origin[2]])

print(f([0.5,0.5,0.5]))
V_tree = Z*Peps(f)

1.8569533817705184


In [ ]:
# Working on Helium with Coulomb direct (CJ) & exchange (CK)
J_n = opr.CouloumbOperator(mra, spinorb_vec, prec)
print(J_n)
K_n = opr.ExchangeOperator(mra, spinorb_vec, prec)
print(K_n)
hdPhi_vec = [] 
vPhi_vec = [] 
orb_energy_vec = []

In [ ]:
def calc_overlap(Bra,Ket):
     S_m = np.empty((len(Bra), len(Ket)))
     for i in range(len(Bra)):
        for j in range(len(Ket)):
            S_m[i, j] = (Bra[i].dot(Ket[j])[0])
     return S_m

In [ ]:
# SCF loop
orbital_error = 1

for i in range(spinorb_vec.shape[0]):
    hdPhi_vec.append(orb.apply_dirac_hamiltonian(spinorb_vec[i], prec, 0.0))

    vPhi_vec.append(orb.apply_potential(-1.0, V_tree, spinorb_vec[i], prec))
print(hdPhi_vec)
print(vPhi_vec)

In [ ]:
hdPhi_vec = np.array(hdPhi_vec)
print(hdPhi_vec)
vPhi_vec = np.array(vPhi_vec)
print(vPhi_vec)

In [ ]:
JPhi_vec = J_n(spinorb_vec)
print(JPhi_vec)
KPhi_vec = K_n(spinorb_vec)
print(KPhi_vec)

In [ ]:
# Definition of full 4c hamitoninan
add_psi_vec = hdPhi_vec + vPhi_vec + JPhi_vec - KPhi_vec
print(add_psi_vec)

In [ ]:
for i in range(spinorb_vec.shape[0]):
    orb_e, imag_e = spinorb_vec[i].dot(add_psi_vec[i])
    orb_energy_vec.append(orb_e)
print(orb_energy_vec)

In [ ]:
orb_energy_vec = np.array(orb_energy_vec)   
print('Orb_Energy', (orb_energy_vec))

In [ ]:
E_tot_C = 0 
for i in range(0, spinorb_vec.shape[0]):     
    E_tot_C += orb_energy_vec[i]
print("E_total(2CJ-CK)", E_tot_C)

In [ ]:
#Calculating of Fij with i /= j  
Fij_matrix = calc_overlap(spinorb_vec, add_psi_vec)
print(Fij_matrix)
Lambda_n = np.diag(np.diag(Fij_matrix))
print(Lambda_n)




In [ ]:
# Calculation of necessary potential contributions to Hellmotz
tmp = add_psi_vec + (Lambda_n - Fij_matrix) @ spinorb_vec

print(tmp)
tmp2 = []
for i in range(0, tmp.shape[0]): 
    tmp2.append(orb.apply_helmholtz(tmp[i], orb_energy_vec[i], c, prec))

tmp2 =  np.array(tmp2)
print(tmp2)

In [ ]:
new_orbital_vec = []
for i in range(spinorb_vec.shape[0]):
    new_orbital_vec.append(orb.apply_dirac_hamiltonian(tmp2[i], prec, orb_energy_vec[i]))
    new_orbital_vec[i].normalize()

new_orbital_vec = np.array(new_orbital_vec)
print(new_orbital_vec)


In [ ]:
# Compute overlap matrices
S_tilde = calc_overlap(new_orbital_vec, new_orbital_vec)
# Löwdin orthonormalization S^{-1/2} = U * Sigma^{-1/2} * U^T
sigma, U = LA.eig(S_tilde)
Sm5 = U @ np.diag(sigma**(-0.5)) @ U.transpose()
spinorb_vec = Sm5 @  new_orbital_vec
print(spinorb_vec)

In [ ]:
INIZIO PORCA TROIA

In [6]:
# Working on Helium with Coulomb direct (CJ) & exchange (CK)
J_n = opr.CouloumbOperator(mra, spinorb_vec, prec)
K_n = opr.ExchangeOperator(mra, spinorb_vec, prec)


def calc_overlap(Bra,Ket):
     S_m = np.empty((len(Bra), len(Ket)))
     for i in range(len(Bra)):
        for j in range(len(Ket)):
            S_m[i, j] = (Bra[i].dot(Ket[j])[0])
     return S_m


# SCF loop
orbital_error = 1
while orbital_error > prec:
     
    hdPhi_vec = [] 
    vPhi_vec = [] 
    orb_energy_vec = []
    orbital_error_vec = []

    for i in range(spinorb_vec.shape[0]):
        hdPhi_vec.append(orb.apply_dirac_hamiltonian(spinorb_vec[i], prec, 0.0))
        vPhi_vec.append(orb.apply_potential(-1.0, V_tree, spinorb_vec[i], prec))
    hdPhi_vec = np.array(hdPhi_vec)
    vPhi_vec = np.array(vPhi_vec)

    JPhi_vec = J_n(spinorb_vec)
    KPhi_vec = K_n(spinorb_vec)
    
    # Definition of full 4c hamitoninan
    add_psi_vec = hdPhi_vec + vPhi_vec + JPhi_vec - KPhi_vec
    
    for i in range(spinorb_vec.shape[0]):
        orb_e, imag_e = spinorb_vec[i].dot(add_psi_vec[i])
        orb_energy_vec.append(orb_e)
    orb_energy_vec = np.array(orb_energy_vec)   
    print('Orb_Energy', (orb_energy_vec))  
    
    E_tot_C = 0 
    for i in range(0, spinorb_vec.shape[0]):     
        E_tot_C += orb_energy_vec[i]
    print("E_total(2CJ-CK)", E_tot_C)
    
    
    #Calculating of Fij with i /= j  and Lamda matrix
    Fij_matrix = calc_overlap(spinorb_vec, add_psi_vec)
    Lambda_n = np.diag(np.diag(Fij_matrix))
    
    
    # Calculation of necessary potential contributions to Hellmotz
    tmp = add_psi_vec + (Lambda_n - Fij_matrix) @ spinorb_vec
    
    tmp2 = []
    for i in range(0, tmp.shape[0]): 
        tmp2.append(orb.apply_helmholtz(tmp[i], orb_energy_vec[i], c, prec))
    tmp2 =  np.array(tmp2)
    
    new_orbital_vec = []
    for i in range(spinorb_vec.shape[0]):
        new_orbital_vec.append(orb.apply_dirac_hamiltonian(tmp2[i], prec, orb_energy_vec[i]))
        new_orbital_vec[i].normalize()    
    new_orbital_vec = np.array(new_orbital_vec)


    delta_psi = new_orbital_vec - spinorb_vec
    for i in range(spinorb_vec.shape[0]):
        orbital_error, imag = delta_psi[i].dot(delta_psi[i])
        orbital_error_vec.append(orbital_error)
    orbital_error_vec = np.array(orbital_error_vec)
    print('Error_1', orbital_error_vec)


    # Compute overlap matrices
    S_tilde = calc_overlap(new_orbital_vec, new_orbital_vec)
    
    # Löwdin orthonormalization S^{-1/2} = U * Sigma^{-1/2} * U^T
    sigma, U = LA.eig(S_tilde)
    Sm5 = U @ np.diag(sigma**(-0.5)) @ U.transpose()
    spinorb_vec = Sm5 @  new_orbital_vec


for i in range(spinorb_vec.shape[0]):
    hdPhi_vec.append(orb.apply_dirac_hamiltonian(spinorb_vec[i], prec, 0.0))
    
    vPhi_vec.append(orb.apply_potential(-1.0, V_tree, spinorb_vec[i], prec))   
hdPhi_vec = np.array(hdPhi_vec)
vPhi_vec = np.array(vPhi_vec)

JPhi_vec = J_n(spinorb_vec)
KPhi_vec = K_n(spinorb_vec)

# Definition of full 4c hamitoninan
add_psi_vec = hdPhi_vec + vPhi_vec + JPhi_vec - KPhi_vec

for i in range(spinorb_vec.shape[0]):
    orb_e, imag_e = spinorb_vec[i].dot(add_psi_vec[i])
    orb_energy_vec.append(orb_e)

orb_energy_vec = np.array(orb_energy_vec)   
print('Orb_Energy', orb_energy_vec)

E_tot_C = 0 
for i in range(0, spinorb_vec.shape[0]):     
    E_tot_C += orb_energy_vec[i]
print("E_total(2CJ-CK)", E_tot_C)

real non zero
imag non zero
real non zero
imag non zero
real non zero
real non zero
imag non zero
imag non zero
real non zero
imag non zero
real non zero
imag non zero
real non zero
imag non zero
real non zero
imag non zero
real non zero
real non zero
imag non zero
imag non zero
real non zero
real non zero
imag non zero
imag non zero
Orb_Energy [18768.4066986 18768.4066986]
E_total(2CJ-CK) 37536.81339720132
mu 1.0893043853780873
mu 1.0893043853780873
mu 1.0893043853780873
mu 1.0893043853780873
mu 1.0893043853824602
mu 1.0893043853824602
mu 1.0893043853824602
mu 1.0893043853824602
Error_1 [3.78580218 3.78580218]


In [ ]:
FINE PORCA TROIA

In [ ]:
# Working on Helium with Dirac-Coulomb-Gaunt Hamiltonian 

orbital_error = 1
while orbital_error > prec:

    #Definiton of Dirac Hamiltonian for alpha component   
    hd_psi_1 = orb.apply_dirac_hamiltonian(spinorb1, prec, 0.0)
    #Definiton of Dirac Hamiltonian for beta component 
    hd_psi_2 = orb.apply_dirac_hamiltonian(spinorb2, prec, 0.0)


    #Definition of alpha density
    n_alpha = spinorb1.density(prec)
    #Definition of beta density
    n_beta = spinorb2.density(prec)

    # Coulomb: Direct (CJ) and Exchange (CK) 

    #Definition of Poisson operator 
    Pua = vp.PoissonOperator(mra,prec=0.000001)
    
    #Defintion of CJ_alpha
    CJ1_term = Pua(n_alpha)*(4*np.pi)
    #Definiton of CJ_beta
    CJ2_term = Pua(n_beta)*(4*np.pi)
    #Definition of Energy Hartree for alpha component
    CJ1_energy = vp.dot(n_alpha, CJ2_term)
    #Definition of Energy Hartree for beta component
    CJ2_energy = vp.dot(n_beta, CJ1_term)
    print("CJ1_energy", CJ1_energy)
    print("CJ2_energy", CJ2_energy)

    #Definition of pseudo density made by differnt spin orbitals
    n_12 = spinorb1.exchange(spinorb2, prec)
    n_21 = spinorb2.exchange(spinorb1, prec)

    #Defintion of CK
    CK1_term = Pua(n_21)*(4*np.pi)
    CK2_term = Pua(n_12)*(4*np.pi)

    #Definition of Exchange energy term for each spin orbital
    CK1_energy = vp.dot(n_12, CK1_term)      
    CK2_energy = vp.dot(n_21, CK2_term)
    print("CK1_energy", CK1_energy)
    print("CK2_energy", CK2_energy)


    # Gaunt: Direct (GJ) and Exchange (GK)
    
    #Definition of alpha(orbital)
    alpha1 = orb.Gaunt(spinorb1, prec)
    alpha2 = orb.Gaunt(spinorb2, prec) 
    
    #Definition of orbital * alpha(orbital)
    spinorb1_alpha1 = spinorb1.exchange(alpha1, prec)
    spinorb2_alpha1 = spinorb2.exchange(alpha1, prec)  
    spinorb1_alpha2 = spinorb1.exchange(alpha2, prec)
    spinorb2_alpha2 = spinorb2.exchange(alpha2, prec) 
     
    #Defintion of GJ
    GJ1_term = Pua(spinorb1_alpha1)*(4*np.pi)
    GJ1_term = GJ1_term / c
    GJ2_term = Pua(spinorb2_alpha2)*(4*np.pi)
    GJ2_term = GJ2_term / c

    #Definition of GJ energy term for each spin orbital
    GJ1_energy = vp.dot(spinorb2_alpha2, GJ1_term)        
    GJ2_energy = vp.dot(spinorb1_alpha1, GJ2_term)
    print("GJ1_energy", GJ1_energy)
    print("GJ2_energy", GJ2_energy)

    #Defintion of GK
    GK1_term = Pua(spinorb2_alpha1)*(4*np.pi)
    GK1_term = GK1_term / c 
    GK2_term = Pua(spinorb1_alpha2)*(4*np.pi)
    GK2_term = GK2_term / c

    #Definition of GK energy term for each spin orbital
    GK1_energy = vp.dot(spinorb1_alpha2, GK1_term)    
    GK2_energy = vp.dot(spinorb2_alpha1, GK2_term)
    print("GK1_energy", GK1_energy)
    print("GK2_energy", GK2_energy)    

    
    #Applying nuclear potential to spin orbital 
    v_psi_1 = orb.apply_potential(-1.0, V_tree, spinorb1, prec)
    v_psi_2 = orb.apply_potential(-1.0, V_tree, spinorb2, prec)
    

    #Definition of full 4c hamitoninan 
    add_psi_1 = hd_psi_1 + v_psi_1 
    add_psi_2 = hd_psi_2 + v_psi_2
    

    #Orbital Energy calculation
    energy_1, imag_1 = spinorb1.dot(add_psi_1)
    energy_2, imag_2 = spinorb2.dot(add_psi_2)
    energy_1 = energy_1 + CJ1_energy - GJ1_energy
    energy_2 = energy_2 + CJ2_energy - GJ2_energy
    print('Energy_1', energy_1, imag_1)
    print('Energy_2', energy_2, imag_2)
    
    #Total Energy with Dirac-Coulomb-Gaunt Hamiltonian 
    E_tot_DCG = energy_1 + energy_2 - CK1_energy + GK1_energy
    print("E_total(Dirac-Coulomb-Gaunt) approximiation", E_tot_DCG)

    
    #energy_12, imag_12 = spinorb1.dot(add_psi_2)
    #F12_spin = energy_12 + 1j * imag_12
    #energy_21, imag_21 = spinorb2.dot(add_psi_1)
    #F21_spin = energy_21 + 1j * imag_21
    #print("F12", energy_12)
    #print("F21", energy_21)

    #Calculation of necessary potential contributions to Hellmotz 
    CJ1_spinorb1 = orb.apply_potential(1.0, CJ1_term, spinorb1, prec)
    GJ1_spinorb1 = orb.apply_potential(1.0, GJ1_term, spinorb1, prec)
    CK1_term_spinorb2 = orb.apply_potential(1.0, CK1_term, spinorb2, prec)
    GK1_term_spinorb2 = orb.apply_potential(1.0, GK1_term, spinorb2, prec)
    #F12_spinorb2 = energy_12 * spinorb2
    V_2J_K_spinorb1 = v_psi_1 + 2*CJ1_spinorb1 - 2*GJ1_spinorb1 - CK1_term_spinorb2 + GK1_term_spinorb2 #- F12_spinorb2
     

    CJ2_spinorb2 = orb.apply_potential(1.0, CJ2_term, spinorb2, prec)
    GJ2_spinorb2 = orb.apply_potential(1.0, GJ2_term, spinorb2, prec)
    CK2_term_spinorb1 = orb.apply_potential(1.0, CK2_term, spinorb1, prec)
    GK2_term_spinorb1 = orb.apply_potential(1.0, GK2_term, spinorb1, prec)
    #F21_spinorb2 = energy_21 * spinorb1
    V_2J_K_spinorb2 = v_psi_2 + 2*CJ2_spinorb2 - 2*GJ2_spinorb2 - CK2_term_spinorb1 + GK2_term_spinorb1 #- F21_spinorb2


    #Calculation of Helmotz
    tmp_1 = orb.apply_helmholtz(V_2J_K_spinorb1, energy_1, c, prec)
    tmp_2 = orb.apply_helmholtz(V_2J_K_spinorb2, energy_2, c, prec)

    new_orbital_1 = orb.apply_dirac_hamiltonian(tmp_1, prec, energy_1)
    new_orbital_1.normalize()
    new_orbital_2 = orb.apply_dirac_hamiltonian(tmp_2, prec, energy_2)
    new_orbital_2.normalize()

    delta_psi_1 = new_orbital_1 - spinorb1
    orbital_error, imag_1 = delta_psi_1.dot(delta_psi_1)
    print('Error_1', orbital_error, imag_1)
    spinorb1 = new_orbital_1

    delta_psi_2 = new_orbital_2 - spinorb2
    orbital_error, imag_2 = delta_psi_2.dot(delta_psi_2)
    print('Error_2', orbital_error, imag_2)
    spinorb2 = new_orbital_2


#Definiton of Dirac Hamiltonian for alpha component   
hd_psi_1 = orb.apply_dirac_hamiltonian(spinorb1, prec, shift=0)
#Definiton of Dirac Hamiltonian for beta component 
hd_psi_2 = orb.apply_dirac_hamiltonian(spinorb2, prec, shift=0)


#Definition of alpha density
n_alpha = spinorb1.density(prec)
#Definition of beta density
n_beta = spinorb2.density(prec)

# Coulomb: Direct (CJ) and Exchange (CK) 

#Definition of Poisson operator 
Pua = vp.PoissonOperator(mra,prec=0.000001)

#Defintion of CJ_alpha
CJ1_term = Pua(n_alpha)*(4*np.pi)
#Definiton of CJ_beta
CJ2_term = Pua(n_beta)*(4*np.pi)
#Definition of Energy Hartree for alpha component
CJ1_energy = vp.dot(n_alpha, CJ2_term)
#Definition of Energy Hartree for beta component
CJ2_energy = vp.dot(n_beta, CJ1_term)
print("CJ1_energy", CJ1_energy)
print("CJ2_energy", CJ2_energy)

#Definition of pseudo density made by differnt spin orbitals
n_12 = spinorb1.exchange(spinorb2, prec)
n_21 = spinorb2.exchange(spinorb1, prec)

#Defintion of CK
CK1_term = Pua(n_21)*(4*np.pi)
CK2_term = Pua(n_12)*(4*np.pi)

#Definition of Exchange energy term for each spin orbital
CK1_energy = vp.dot(n_12, CK1_term)      
CK2_energy = vp.dot(n_21, CK2_term)
print("CK1_energy", CK1_energy)
print("CK2_energy", CK2_energy)


# Gaunt: Direct (GJ) and Exchange (GK)

#Definition of alpha(orbital)
alpha1 = orb.Gaunt(spinorb1, prec)
alpha2 = orb.Gaunt(spinorb2, prec) 

#Definition of orbital * alpha(orbital)
spinorb1_alpha1 = spinorb1.exchange(alpha1, prec)
spinorb2_alpha1 = spinorb2.exchange(alpha1, prec)  
spinorb1_alpha2 = spinorb1.exchange(alpha2, prec)
spinorb2_alpha2 = spinorb2.exchange(alpha2, prec) 
    
#Defintion of GJ
GJ1_term = Pua(spinorb1_alpha1)*(4*np.pi)
GJ2_term = Pua(spinorb2_alpha2)*(4*np.pi)

#Definition of GJ energy term for each spin orbital
GJ1_energy = vp.dot(spinorb2_alpha2, GJ1_term)
GJ1_energy = GJ1_energy / c     
GJ2_energy = vp.dot(spinorb1_alpha1, GJ2_term)
GJ2_energy = GJ2_energy / c
print("GJ1_energy", GJ1_energy)
print("GJ2_energy", GJ2_energy)

#Defintion of GK
GK1_term = Pua(spinorb2_alpha1)*(4*np.pi)
GK2_term = Pua(spinorb1_alpha2)*(4*np.pi)

#Definition of GK energy term for each spin orbital
GK1_energy = vp.dot(spinorb1_alpha2, GK1_term)
GK1_energy = GK1_energy / c      
GK2_energy = vp.dot(spinorb2_alpha1, GK2_term)
GK2_energy = GK2_energy / c 
print("GK1_energy", GK1_energy)
print("GK2_energy", GK2_energy)    


#Applying nuclear potential to spin orbital 
v_psi_1 = orb.apply_potential(-1.0, V_tree, spinorb1, prec)
v_psi_2 = orb.apply_potential(-1.0, V_tree, spinorb2, prec)
    

#Definition of full 4c hamitoninan 
add_psi_1 = hd_psi_1 + v_psi_1
add_psi_2 = hd_psi_2 + v_psi_2
    

#Orbital Energy calculation
energy_1, imag_1 = spinorb1.dot(add_psi_1)
energy_2, imag_2 = spinorb2.dot(add_psi_2)
energy_1 = energy_1 + CJ1_energy - GJ1_energy
energy_2 = energy_2 + CJ2_energy - GJ2_energy
print('Energy_1', energy_1, imag_1)
print('Energy_2', energy_2, imag_2)
    
#Total Energy with Dirac-Coulomb-Gaunt Hamiltonian 
E_tot_DCG = energy_1 + energy_2 - CK1_energy + GK1_energy
print("E_total(Dirac-Coulomb-Gaunt) approximiation", E_tot_DCG)